# Tensors and Operations: PyTorch vs TensorFlow

**Learning Objectives:**
- Master tensor creation, manipulation, and operations in both frameworks
- Understand device management (CPU/GPU) differences
- Compare performance and memory usage patterns
- Learn when to choose each framework for tensor operations

**Prerequisites:** NumPy essentials, data preparation basics

**Estimated Time:** 45 minutes

---

Tensors are the fundamental data structure in both PyTorch and TensorFlow. Understanding how each framework handles tensors is crucial for:
- Efficient model implementation
- Memory management
- Performance optimization
- Debugging and development

This notebook provides side-by-side comparisons to help you understand the similarities and differences.

In [ ]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

# Add src to path for our utilities
sys.path.append(os.path.join('..', '..', 'src'))

# Import our comparison utilities
from utils.comparison_tools import FrameworkComparison, create_side_by_side_comparison
from utils.visualization import FrameworkVisualizer

# Try to import frameworks
try:
    import torch
    PYTORCH_AVAILABLE = True
    print(f"✅ PyTorch {torch.__version__} available")

    # Check for GPU
    if torch.cuda.is_available():
        print(f"   🚀 CUDA available: {torch.cuda.get_device_name(0)}")
    else:
        print("   💻 CPU only")

except ImportError:
    PYTORCH_AVAILABLE = False
    print("❌ PyTorch not available")

try:
    import tensorflow as tf
    TENSORFLOW_AVAILABLE = True
    print(f"✅ TensorFlow {tf.__version__} available")

    # Check for GPU
    if tf.config.list_physical_devices('GPU'):
        print(f"   🚀 GPU available: {len(tf.config.list_physical_devices('GPU'))} device(s)")
    else:
        print("   💻 CPU only")

except ImportError:
    TENSORFLOW_AVAILABLE = False
    print("❌ TensorFlow not available")

# Set random seeds for reproducibility
np.random.seed(42)
if PYTORCH_AVAILABLE:
    torch.manual_seed(42)
if TENSORFLOW_AVAILABLE:
    tf.random.set_seed(42)

print(f"\nNumPy version: {np.__version__}")

## 1. Tensor Creation

Understanding different ways to create tensors in both frameworks.

In [ ]:
print("=" * 60)
print("TENSOR CREATION COMPARISON")
print("=" * 60)

# Create comparison framework
comparator = FrameworkComparison()

# Example 1: Creating tensors from lists
print("\n1. Creating tensors from lists:")

data_list = [[1, 2, 3], [4, 5, 6]]

def create_pytorch_from_list():
    if PYTORCH_AVAILABLE:
        return torch.tensor(data_list, dtype=torch.float32)
    return None

def create_tensorflow_from_list():
    if TENSORFLOW_AVAILABLE:
        return tf.constant(data_list, dtype=tf.float32)
    return None

result = comparator.compare_implementations(
    pytorch_func=create_pytorch_from_list,
    tensorflow_func=create_tensorflow_from_list,
    name="tensor_from_list"
)

if result['pytorch']['success'] and result['tensorflow']['success']:
    pt_tensor = result['pytorch']['output']
    tf_tensor = result['tensorflow']['output']

    print(f"PyTorch: {pt_tensor.shape}, dtype: {pt_tensor.dtype}")
    print(f"TensorFlow: {tf_tensor.shape}, dtype: {tf_tensor.dtype}")
    print(f"Values match: {result['comparison']['outputs_equal']}")

# Show side-by-side code
pytorch_code = """
import torch
data = [[1, 2, 3], [4, 5, 6]]
tensor = torch.tensor(data, dtype=torch.float32)
print(f"Shape: {tensor.shape}")
print(f"Dtype: {tensor.dtype}")
"""

tensorflow_code = """
import tensorflow as tf
data = [[1, 2, 3], [4, 5, 6]]
tensor = tf.constant(data, dtype=tf.float32)
print(f"Shape: {tensor.shape}")
print(f"Dtype: {tensor.dtype}")
"""

print(create_side_by_side_comparison(pytorch_code, tensorflow_code, "Creating Tensors from Lists"))

In [ ]:
# Example 2: Creating special tensors (zeros, ones, random)
print("\n2. Creating special tensors:")

shape = (3, 4)

# Zeros
def create_pytorch_zeros():
    if PYTORCH_AVAILABLE:
        return torch.zeros(shape, dtype=torch.float32)
    return None

def create_tensorflow_zeros():
    if TENSORFLOW_AVAILABLE:
        return tf.zeros(shape, dtype=tf.float32)
    return None

zeros_result = comparator.compare_implementations(
    pytorch_func=create_pytorch_zeros,
    tensorflow_func=create_tensorflow_zeros,
    name="zeros_tensor"
)

print(f"Zeros tensor comparison: {zeros_result['comparison'].get('outputs_equal', 'N/A')}")

# Random tensors
def create_pytorch_random():
    if PYTORCH_AVAILABLE:
        torch.manual_seed(42)  # For reproducibility
        return torch.randn(shape, dtype=torch.float32)
    return None

def create_tensorflow_random():
    if TENSORFLOW_AVAILABLE:
        tf.random.set_seed(42)  # For reproducibility
        return tf.random.normal(shape, dtype=tf.float32)
    return None

random_result = comparator.compare_implementations(
    pytorch_func=create_pytorch_random,
    tensorflow_func=create_tensorflow_random,
    name="random_tensor"
)

print(f"Random tensor shapes match: {random_result['comparison'].get('shape_match', 'N/A')}")
print(f"Random values are different (expected): {not random_result['comparison'].get('outputs_equal', True)}")

# Show comprehensive tensor creation methods
print("\n📋 Tensor Creation Methods Comparison:")
creation_methods = {
    "From data": {
        "PyTorch": "torch.tensor(data)",
        "TensorFlow": "tf.constant(data)"
    },
    "Zeros": {
        "PyTorch": "torch.zeros(shape)",
        "TensorFlow": "tf.zeros(shape)"
    },
    "Ones": {
        "PyTorch": "torch.ones(shape)",
        "TensorFlow": "tf.ones(shape)"
    },
    "Random normal": {
        "PyTorch": "torch.randn(shape)",
        "TensorFlow": "tf.random.normal(shape)"
    },
    "Random uniform": {
        "PyTorch": "torch.rand(shape)",
        "TensorFlow": "tf.random.uniform(shape)"
    },
    "Identity": {
        "PyTorch": "torch.eye(n)",
        "TensorFlow": "tf.eye(n)"
    },
    "Range": {
        "PyTorch": "torch.arange(start, end)",
        "TensorFlow": "tf.range(start, limit)"
    }
}

for method, frameworks in creation_methods.items():
    print(f"{method:15} | PyTorch: {frameworks['PyTorch']:25} | TensorFlow: {frameworks['TensorFlow']}")

## 2. Basic Tensor Operations

Comparing fundamental tensor operations between frameworks.

In [ ]:
print("\n" + "=" * 60)
print("BASIC TENSOR OPERATIONS")
print("=" * 60)

# Create sample tensors for operations
np.random.seed(42)
sample_data = np.random.randn(3, 4).astype(np.float32)

if PYTORCH_AVAILABLE:
    pt_tensor = torch.from_numpy(sample_data)
    print(f"PyTorch tensor: {pt_tensor.shape}, dtype: {pt_tensor.dtype}")

if TENSORFLOW_AVAILABLE:
    tf_tensor = tf.constant(sample_data)
    print(f"TensorFlow tensor: {tf_tensor.shape}, dtype: {tf_tensor.dtype}")

# Element-wise operations
print("\n1. Element-wise Operations:")

def pytorch_elementwise_ops():
    if PYTORCH_AVAILABLE:
        x = torch.from_numpy(sample_data)

        # Basic operations
        squared = x ** 2
        sqrt_abs = torch.sqrt(torch.abs(x))
        exp_x = torch.exp(x)

        # Activation functions
        relu = torch.relu(x)
        sigmoid = torch.sigmoid(x)
        tanh = torch.tanh(x)

        return {
            'squared_mean': torch.mean(squared).item(),
            'sqrt_abs_mean': torch.mean(sqrt_abs).item(),
            'exp_mean': torch.mean(exp_x).item(),
            'relu_mean': torch.mean(relu).item(),
            'sigmoid_mean': torch.mean(sigmoid).item(),
            'tanh_mean': torch.mean(tanh).item()
        }
    return None

def tensorflow_elementwise_ops():
    if TENSORFLOW_AVAILABLE:
        x = tf.constant(sample_data)

        # Basic operations
        squared = tf.square(x)
        sqrt_abs = tf.sqrt(tf.abs(x))
        exp_x = tf.exp(x)

        # Activation functions
        relu = tf.nn.relu(x)
        sigmoid = tf.nn.sigmoid(x)
        tanh = tf.nn.tanh(x)

        return {
            'squared_mean': tf.reduce_mean(squared).numpy().item(),
            'sqrt_abs_mean': tf.reduce_mean(sqrt_abs).numpy().item(),
            'exp_mean': tf.reduce_mean(exp_x).numpy().item(),
            'relu_mean': tf.reduce_mean(relu).numpy().item(),
            'sigmoid_mean': tf.reduce_mean(sigmoid).numpy().item(),
            'tanh_mean': tf.reduce_mean(tanh).numpy().item()
        }
    return None

ops_result = comparator.compare_implementations(
    pytorch_func=pytorch_elementwise_ops,
    tensorflow_func=tensorflow_elementwise_ops,
    name="elementwise_operations"
)

if ops_result['pytorch']['success'] and ops_result['tensorflow']['success']:
    pt_results = ops_result['pytorch']['output']
    tf_results = ops_result['tensorflow']['output']

    print("Operation results comparison:")
    for op in pt_results.keys():
        pt_val = pt_results[op]
        tf_val = tf_results[op]
        diff = abs(pt_val - tf_val)
        print(f"{op:15}: PyTorch={pt_val:.6f}, TensorFlow={tf_val:.6f}, diff={diff:.2e}")

# Show side-by-side activation functions
pytorch_activations = """
import torch
x = torch.randn(3, 4)

# Activation functions
relu = torch.relu(x)
sigmoid = torch.sigmoid(x)
tanh = torch.tanh(x)
softmax = torch.softmax(x, dim=1)
log_softmax = torch.log_softmax(x, dim=1)
"""

tensorflow_activations = """
import tensorflow as tf
x = tf.random.normal((3, 4))

# Activation functions
relu = tf.nn.relu(x)
sigmoid = tf.nn.sigmoid(x)
tanh = tf.nn.tanh(x)
softmax = tf.nn.softmax(x, axis=1)
log_softmax = tf.nn.log_softmax(x, axis=1)
"""

print(create_side_by_side_comparison(pytorch_activations, tensorflow_activations, "Activation Functions"))

In [ ]:
# Reduction operations
print("\n2. Reduction Operations:")

def pytorch_reductions():
    if PYTORCH_AVAILABLE:
        x = torch.from_numpy(sample_data)

        return {
            'sum_all': torch.sum(x).item(),
            'mean_all': torch.mean(x).item(),
            'max_all': torch.max(x).item(),
            'min_all': torch.min(x).item(),
            'std_all': torch.std(x).item(),
            'sum_axis0': torch.sum(x, dim=0).tolist(),
            'mean_axis1': torch.mean(x, dim=1).tolist()
        }
    return None

def tensorflow_reductions():
    if TENSORFLOW_AVAILABLE:
        x = tf.constant(sample_data)

        return {
            'sum_all': tf.reduce_sum(x).numpy().item(),
            'mean_all': tf.reduce_mean(x).numpy().item(),
            'max_all': tf.reduce_max(x).numpy().item(),
            'min_all': tf.reduce_min(x).numpy().item(),
            'std_all': tf.math.reduce_std(x).numpy().item(),
            'sum_axis0': tf.reduce_sum(x, axis=0).numpy().tolist(),
            'mean_axis1': tf.reduce_mean(x, axis=1).numpy().tolist()
        }
    return None

reductions_result = comparator.compare_implementations(
    pytorch_func=pytorch_reductions,
    tensorflow_func=tensorflow_reductions,
    name="reduction_operations"
)

if reductions_result['pytorch']['success'] and reductions_result['tensorflow']['success']:
    pt_results = reductions_result['pytorch']['output']
    tf_results = reductions_result['tensorflow']['output']

    print("Reduction operations comparison:")
    for op in ['sum_all', 'mean_all', 'max_all', 'min_all', 'std_all']:
        pt_val = pt_results[op]
        tf_val = tf_results[op]
        diff = abs(pt_val - tf_val)
        print(f"{op:12}: PyTorch={pt_val:.6f}, TensorFlow={tf_val:.6f}, diff={diff:.2e}")

# Show reduction operations side-by-side
pytorch_reductions_code = """
import torch
x = torch.randn(3, 4)

# Global reductions
total_sum = torch.sum(x)
mean_val = torch.mean(x)
max_val = torch.max(x)
std_val = torch.std(x)

# Axis-specific reductions
sum_rows = torch.sum(x, dim=0)    # Sum along rows
mean_cols = torch.mean(x, dim=1)  # Mean along columns

# Keep dimensions
sum_keepdim = torch.sum(x, dim=1, keepdim=True)
"""

tensorflow_reductions_code = """
import tensorflow as tf
x = tf.random.normal((3, 4))

# Global reductions
total_sum = tf.reduce_sum(x)
mean_val = tf.reduce_mean(x)
max_val = tf.reduce_max(x)
std_val = tf.math.reduce_std(x)

# Axis-specific reductions
sum_rows = tf.reduce_sum(x, axis=0)    # Sum along rows
mean_cols = tf.reduce_mean(x, axis=1)  # Mean along columns

# Keep dimensions
sum_keepdim = tf.reduce_sum(x, axis=1, keepdims=True)
"""

print(create_side_by_side_comparison(pytorch_reductions_code, tensorflow_reductions_code, "Reduction Operations"))

## 3. Matrix Operations and Linear Algebra

Comparing linear algebra operations essential for neural networks.

In [ ]:
print("\n" + "=" * 60)
print("MATRIX OPERATIONS AND LINEAR ALGEBRA")
print("=" * 60)

# Create matrices for linear algebra operations
np.random.seed(42)
matrix_a = np.random.randn(4, 3).astype(np.float32)
matrix_b = np.random.randn(3, 5).astype(np.float32)
square_matrix = np.random.randn(4, 4).astype(np.float32)

print(f"Matrix A shape: {matrix_a.shape}")
print(f"Matrix B shape: {matrix_b.shape}")
print(f"Square matrix shape: {square_matrix.shape}")

# Matrix multiplication
print("\n1. Matrix Multiplication:")

def pytorch_matmul():
    if PYTORCH_AVAILABLE:
        A = torch.from_numpy(matrix_a)
        B = torch.from_numpy(matrix_b)

        # Different ways to do matrix multiplication
        result1 = torch.matmul(A, B)
        result2 = A @ B  # Python 3.5+ operator
        result3 = torch.mm(A, B)  # 2D matrices only

        # Batch matrix multiplication
        batch_A = A.unsqueeze(0).repeat(2, 1, 1)  # Shape: (2, 4, 3)
        batch_B = B.unsqueeze(0).repeat(2, 1, 1)  # Shape: (2, 3, 5)
        batch_result = torch.bmm(batch_A, batch_B)  # Shape: (2, 4, 5)

        return {
            'matmul_result': result1,
            'results_equal': torch.allclose(result1, result2) and torch.allclose(result1, result3),
            'batch_shape': batch_result.shape
        }
    return None

def tensorflow_matmul():
    if TENSORFLOW_AVAILABLE:
        A = tf.constant(matrix_a)
        B = tf.constant(matrix_b)

        # Different ways to do matrix multiplication
        result1 = tf.matmul(A, B)
        result2 = A @ B  # Python 3.5+ operator
        result3 = tf.linalg.matmul(A, B)

        # Batch matrix multiplication
        batch_A = tf.expand_dims(A, 0)
        batch_A = tf.tile(batch_A, [2, 1, 1])  # Shape: (2, 4, 3)
        batch_B = tf.expand_dims(B, 0)
        batch_B = tf.tile(batch_B, [2, 1, 1])  # Shape: (2, 3, 5)
        batch_result = tf.matmul(batch_A, batch_B)  # Shape: (2, 4, 5)

        return {
            'matmul_result': result1,
            'results_equal': tf.reduce_all(tf.equal(result1, result2)) and tf.reduce_all(tf.equal(result1, result3)),
            'batch_shape': batch_result.shape
        }
    return None

matmul_result = comparator.compare_implementations(
    pytorch_func=pytorch_matmul,
    tensorflow_func=tensorflow_matmul,
    name="matrix_multiplication"
)

if matmul_result['pytorch']['success'] and matmul_result['tensorflow']['success']:
    pt_results = matmul_result['pytorch']['output']
    tf_results = matmul_result['tensorflow']['output']

    print(f"Matrix multiplication results match: {matmul_result['comparison']['outputs_equal']}")
    print(f"PyTorch different methods equal: {pt_results['results_equal']}")
    print(f"TensorFlow different methods equal: {tf_results['results_equal'].numpy()}")
    print(f"Batch multiplication shapes - PyTorch: {pt_results['batch_shape']}, TensorFlow: {tf_results['batch_shape']}")

# Show matrix multiplication code comparison
pytorch_matmul_code = """
import torch
A = torch.randn(4, 3)
B = torch.randn(3, 5)

# Matrix multiplication methods
result1 = torch.matmul(A, B)  # General
result2 = A @ B               # Operator
result3 = torch.mm(A, B)      # 2D only

# Batch matrix multiplication
batch_A = A.unsqueeze(0).repeat(2, 1, 1)
batch_B = B.unsqueeze(0).repeat(2, 1, 1)
batch_result = torch.bmm(batch_A, batch_B)
"""

tensorflow_matmul_code = """
import tensorflow as tf
A = tf.random.normal((4, 3))
B = tf.random.normal((3, 5))

# Matrix multiplication methods
result1 = tf.matmul(A, B)         # General
result2 = A @ B                   # Operator
result3 = tf.linalg.matmul(A, B)  # Explicit

# Batch matrix multiplication
batch_A = tf.expand_dims(A, 0)
batch_A = tf.tile(batch_A, [2, 1, 1])
batch_B = tf.expand_dims(B, 0)
batch_B = tf.tile(batch_B, [2, 1, 1])
batch_result = tf.matmul(batch_A, batch_B)
"""

print(create_side_by_side_comparison(pytorch_matmul_code, tensorflow_matmul_code, "Matrix Multiplication"))

In [ ]:
# Advanced linear algebra operations
print("\n2. Advanced Linear Algebra:")

def pytorch_linalg():
    if PYTORCH_AVAILABLE:
        # Make square matrix symmetric for stable eigenvalues
        A = torch.from_numpy(square_matrix)
        A_sym = (A + A.T) / 2

        # Linear algebra operations
        det = torch.det(A_sym)
        trace = torch.trace(A_sym)

        # Eigenvalues and eigenvectors
        eigenvals, eigenvecs = torch.linalg.eig(A_sym)
        eigenvals = eigenvals.real  # Take real part

        # SVD
        U, S, Vh = torch.linalg.svd(A_sym)

        # Matrix norms
        frobenius_norm = torch.linalg.norm(A_sym, 'fro')
        spectral_norm = torch.linalg.norm(A_sym, 2)

        return {
            'determinant': det.item(),
            'trace': trace.item(),
            'eigenvals_sum': torch.sum(eigenvals).item(),
            'singular_vals_sum': torch.sum(S).item(),
            'frobenius_norm': frobenius_norm.item(),
            'spectral_norm': spectral_norm.item()
        }
    return None

def tensorflow_linalg():
    if TENSORFLOW_AVAILABLE:
        # Make square matrix symmetric for stable eigenvalues
        A = tf.constant(square_matrix)
        A_sym = (A + tf.transpose(A)) / 2

        # Linear algebra operations
        det = tf.linalg.det(A_sym)
        trace = tf.linalg.trace(A_sym)

        # Eigenvalues and eigenvectors
        eigenvals, eigenvecs = tf.linalg.eig(A_sym)
        eigenvals = tf.math.real(eigenvals)  # Take real part

        # SVD
        S, U, Vh = tf.linalg.svd(A_sym)

        # Matrix norms
        frobenius_norm = tf.linalg.norm(A_sym, 'fro')
        spectral_norm = tf.linalg.norm(A_sym, 2)

        return {
            'determinant': det.numpy().item(),
            'trace': trace.numpy().item(),
            'eigenvals_sum': tf.reduce_sum(eigenvals).numpy().item(),
            'singular_vals_sum': tf.reduce_sum(S).numpy().item(),
            'frobenius_norm': frobenius_norm.numpy().item(),
            'spectral_norm': spectral_norm.numpy().item()
        }
    return None

linalg_result = comparator.compare_implementations(
    pytorch_func=pytorch_linalg,
    tensorflow_func=tensorflow_linalg,
    name="linear_algebra"
)

if linalg_result['pytorch']['success'] and linalg_result['tensorflow']['success']:
    pt_results = linalg_result['pytorch']['output']
    tf_results = linalg_result['tensorflow']['output']

    print("Linear algebra operations comparison:")
    for op in pt_results.keys():
        pt_val = pt_results[op]
        tf_val = tf_results[op]
        diff = abs(pt_val - tf_val)
        print(f"{op:18}: PyTorch={pt_val:.6f}, TensorFlow={tf_val:.6f}, diff={diff:.2e}")

# Linear algebra operations comparison table
print("\n📋 Linear Algebra Operations Comparison:")
linalg_ops = {
    "Matrix multiplication": {
        "PyTorch": "torch.matmul(A, B) or A @ B",
        "TensorFlow": "tf.matmul(A, B) or A @ B"
    },
    "Transpose": {
        "PyTorch": "A.T or torch.transpose(A, 0, 1)",
        "TensorFlow": "tf.transpose(A)"
    },
    "Determinant": {
        "PyTorch": "torch.det(A)",
        "TensorFlow": "tf.linalg.det(A)"
    },
    "Eigenvalues": {
        "PyTorch": "torch.linalg.eig(A)",
        "TensorFlow": "tf.linalg.eig(A)"
    },
    "SVD": {
        "PyTorch": "torch.linalg.svd(A)",
        "TensorFlow": "tf.linalg.svd(A)"
    },
    "Matrix inverse": {
        "PyTorch": "torch.linalg.inv(A)",
        "TensorFlow": "tf.linalg.inv(A)"
    },
    "Matrix norm": {
        "PyTorch": "torch.linalg.norm(A, ord)",
        "TensorFlow": "tf.linalg.norm(A, ord)"
    }
}

for op, frameworks in linalg_ops.items():
    print(f"{op:20} | PyTorch: {frameworks['PyTorch']:30} | TensorFlow: {frameworks['TensorFlow']}")

## 4. Tensor Reshaping and Manipulation

Comparing tensor shape manipulation operations.

In [ ]:
print("\n" + "=" * 60)
print("TENSOR RESHAPING AND MANIPULATION")
print("=" * 60)

# Create sample tensor for reshaping
np.random.seed(42)
sample_tensor_data = np.random.randn(2, 3, 4).astype(np.float32)

print(f"Original tensor shape: {sample_tensor_data.shape}")

def pytorch_reshaping():
    if PYTORCH_AVAILABLE:
        x = torch.from_numpy(sample_tensor_data)

        # Reshaping operations
        reshaped = x.reshape(6, 4)  # or x.view(6, 4)
        flattened = x.flatten()  # or x.reshape(-1)

        # Dimension manipulation
        x.unsqueeze(0).squeeze(0)  # Add and remove dimension
        transposed = x.transpose(0, 2)  # Swap dimensions 0 and 2
        permuted = x.permute(2, 0, 1)  # Reorder all dimensions

        # Concatenation and stacking
        concat_dim0 = torch.cat([x, x], dim=0)  # Shape: (4, 3, 4)
        stack_new_dim = torch.stack([x, x], dim=0)  # Shape: (2, 2, 3, 4)

        # Splitting
        split_tensors = torch.split(x, 1, dim=0)  # Split into tensors of size 1 along dim 0
        chunk_tensors = torch.chunk(x, 2, dim=1)  # Split into 2 chunks along dim 1

        return {
            'reshaped_shape': reshaped.shape,
            'flattened_shape': flattened.shape,
            'transposed_shape': transposed.shape,
            'permuted_shape': permuted.shape,
            'concat_shape': concat_dim0.shape,
            'stack_shape': stack_new_dim.shape,
            'num_splits': len(split_tensors),
            'num_chunks': len(chunk_tensors),
            'chunk_shapes': [chunk.shape for chunk in chunk_tensors]
        }
    return None

def tensorflow_reshaping():
    if TENSORFLOW_AVAILABLE:
        x = tf.constant(sample_tensor_data)

        # Reshaping operations
        reshaped = tf.reshape(x, (6, 4))
        flattened = tf.reshape(x, (-1,))  # or tf.keras.layers.Flatten()(x)

        # Dimension manipulation
        tf.squeeze(tf.expand_dims(x, 0), 0)  # Add and remove dimension
        transposed = tf.transpose(x, perm=[2, 1, 0])  # Reorder dimensions
        permuted = tf.transpose(x, perm=[2, 0, 1])  # Reorder all dimensions

        # Concatenation and stacking
        concat_dim0 = tf.concat([x, x], axis=0)  # Shape: (4, 3, 4)
        stack_new_dim = tf.stack([x, x], axis=0)  # Shape: (2, 2, 3, 4)

        # Splitting
        split_tensors = tf.split(x, num_or_size_splits=2, axis=0)  # Split into 2 parts along dim 0

        return {
            'reshaped_shape': reshaped.shape,
            'flattened_shape': flattened.shape,
            'transposed_shape': transposed.shape,
            'permuted_shape': permuted.shape,
            'concat_shape': concat_dim0.shape,
            'stack_shape': stack_new_dim.shape,
            'num_splits': len(split_tensors),
            'split_shapes': [tensor.shape for tensor in split_tensors]
        }
    return None

reshaping_result = comparator.compare_implementations(
    pytorch_func=pytorch_reshaping,
    tensorflow_func=tensorflow_reshaping,
    name="tensor_reshaping"
)

if reshaping_result['pytorch']['success'] and reshaping_result['tensorflow']['success']:
    pt_results = reshaping_result['pytorch']['output']
    tf_results = reshaping_result['tensorflow']['output']

    print("Tensor reshaping operations comparison:")
    print(f"Reshaped shape - PyTorch: {pt_results['reshaped_shape']}, TensorFlow: {tf_results['reshaped_shape']}")
    print(f"Flattened shape - PyTorch: {pt_results['flattened_shape']}, TensorFlow: {tf_results['flattened_shape']}")
    print(f"Transposed shape - PyTorch: {pt_results['transposed_shape']}, TensorFlow: {tf_results['transposed_shape']}")
    print(f"Permuted shape - PyTorch: {pt_results['permuted_shape']}, TensorFlow: {tf_results['permuted_shape']}")
    print(f"Concatenated shape - PyTorch: {pt_results['concat_shape']}, TensorFlow: {tf_results['concat_shape']}")
    print(f"Stacked shape - PyTorch: {pt_results['stack_shape']}, TensorFlow: {tf_results['stack_shape']}")

# Show reshaping operations side-by-side
pytorch_reshape_code = """
import torch
x = torch.randn(2, 3, 4)

# Reshaping
reshaped = x.reshape(6, 4)  # or x.view(6, 4)
flattened = x.flatten()     # or x.reshape(-1)

# Dimension manipulation
unsqueezed = x.unsqueeze(0)      # Add dimension
squeezed = unsqueezed.squeeze(0)  # Remove dimension
transposed = x.transpose(0, 2)    # Swap dims
permuted = x.permute(2, 0, 1)     # Reorder dims

# Concatenation and stacking
concat = torch.cat([x, x], dim=0)
stack = torch.stack([x, x], dim=0)
"""

tensorflow_reshape_code = """
import tensorflow as tf
x = tf.random.normal((2, 3, 4))

# Reshaping
reshaped = tf.reshape(x, (6, 4))
flattened = tf.reshape(x, (-1,))

# Dimension manipulation
expanded = tf.expand_dims(x, 0)      # Add dimension
squeezed = tf.squeeze(expanded, 0)   # Remove dimension
transposed = tf.transpose(x, [2, 1, 0])  # Reorder dims
permuted = tf.transpose(x, [2, 0, 1])    # Reorder dims

# Concatenation and stacking
concat = tf.concat([x, x], axis=0)
stack = tf.stack([x, x], axis=0)
"""

print(create_side_by_side_comparison(pytorch_reshape_code, tensorflow_reshape_code, "Tensor Reshaping"))

## 5. Performance Benchmarking

Comparing the performance of tensor operations between frameworks.

In [ ]:
print("\n" + "=" * 60)
print("PERFORMANCE BENCHMARKING")
print("=" * 60)

# Create larger tensors for meaningful benchmarks
large_size = (1000, 1000)
np.random.seed(42)
large_data_a = np.random.randn(*large_size).astype(np.float32)
large_data_b = np.random.randn(*large_size).astype(np.float32)

print(f"Benchmarking with tensors of shape: {large_size}")
print(f"Memory per tensor: {large_data_a.nbytes / 1024 / 1024:.2f} MB")

# Benchmark matrix multiplication
def pytorch_matmul_benchmark():
    if PYTORCH_AVAILABLE:
        A = torch.from_numpy(large_data_a)
        B = torch.from_numpy(large_data_b)

        # Warm up
        for _ in range(3):
            _ = torch.matmul(A, B)

        # Actual benchmark
        start_time = time.time()
        result = torch.matmul(A, B)
        end_time = time.time()

        return {
            'execution_time': end_time - start_time,
            'result_shape': result.shape,
            'result_mean': torch.mean(result).item()
        }
    return None

def tensorflow_matmul_benchmark():
    if TENSORFLOW_AVAILABLE:
        A = tf.constant(large_data_a)
        B = tf.constant(large_data_b)

        # Warm up
        for _ in range(3):
            _ = tf.matmul(A, B)

        # Actual benchmark
        start_time = time.time()
        result = tf.matmul(A, B)
        end_time = time.time()

        return {
            'execution_time': end_time - start_time,
            'result_shape': result.shape,
            'result_mean': tf.reduce_mean(result).numpy().item()
        }
    return None

# Run benchmark
benchmark_result = comparator.benchmark_performance(
    pytorch_func=pytorch_matmul_benchmark,
    tensorflow_func=tensorflow_matmul_benchmark,
    num_runs=5,
    warmup_runs=2
)

print("\nMatrix Multiplication Benchmark Results:")
if benchmark_result['pytorch'].get('success'):
    pt_time = benchmark_result['pytorch']['mean_time']
    print(f"PyTorch: {pt_time:.4f}s (±{benchmark_result['pytorch']['std_time']:.4f}s)")

if benchmark_result['tensorflow'].get('success'):
    tf_time = benchmark_result['tensorflow']['mean_time']
    print(f"TensorFlow: {tf_time:.4f}s (±{benchmark_result['tensorflow']['std_time']:.4f}s)")

if 'relative_performance' in benchmark_result:
    rel_perf = benchmark_result['relative_performance']
    faster_framework = rel_perf['faster_framework']
    speedup = rel_perf['speedup_ratio']
    print(f"\n🏆 {faster_framework.capitalize()} is {speedup:.2f}x faster")

# Visualize benchmark results if we have data
if benchmark_result['pytorch'].get('success') or benchmark_result['tensorflow'].get('success'):
    visualizer = FrameworkVisualizer()
    fig = visualizer.plot_performance_comparison(
        benchmark_result,
        title="Matrix Multiplication Performance"
    )
    plt.show()

## Summary and Key Takeaways

**What we've learned:**

1. **Tensor Creation**: Both frameworks offer similar tensor creation methods with slight API differences
2. **Basic Operations**: Element-wise and reduction operations are nearly identical in functionality
3. **Linear Algebra**: Both provide comprehensive linear algebra operations with similar performance
4. **Reshaping**: Tensor manipulation operations exist in both, with different naming conventions
5. **Performance**: Performance differences are often minimal and depend on specific operations and hardware

**Key Differences:**

| Aspect | PyTorch | TensorFlow |
|--------|---------|------------|
| **Tensor Creation** | `torch.tensor()`, `torch.zeros()` | `tf.constant()`, `tf.zeros()` |
| **Reshaping** | `.reshape()`, `.view()` | `tf.reshape()` |
| **Dimension Ops** | `.unsqueeze()`, `.squeeze()` | `tf.expand_dims()`, `tf.squeeze()` |
| **Reductions** | `torch.sum()`, `torch.mean()` | `tf.reduce_sum()`, `tf.reduce_mean()` |
| **Axis Parameter** | `dim=` | `axis=` |
| **Transpose** | `.T`, `.transpose()` | `tf.transpose()` |
| **Random Ops** | `torch.randn()`, `torch.rand()` | `tf.random.normal()`, `tf.random.uniform()` |

**Best Practices:**

**PyTorch:**
- Use `.view()` when you're sure about memory layout, `.reshape()` otherwise
- Prefer `torch.matmul()` or `@` operator for matrix multiplication
- Use `.item()` to extract scalar values from tensors
- Leverage broadcasting for efficient operations

**TensorFlow:**
- Use `tf.reshape()` for all reshaping operations
- Prefer `tf.matmul()` or `@` operator for matrix multiplication
- Use `.numpy()` to convert to NumPy when needed
- Remember that TensorFlow operations return new tensors (immutable)

**Performance Considerations:**
- Both frameworks are highly optimized for tensor operations
- Performance differences are usually small for basic operations
- GPU acceleration provides similar benefits in both frameworks
- Choose based on ecosystem and development preferences, not just raw performance

**When to Choose What:**
- **PyTorch**: Research, prototyping, dynamic models, easier debugging
- **TensorFlow**: Production deployment, mobile/edge devices, large-scale serving

**Next Steps:**
- Learn about computational graphs and automatic differentiation
- Explore GPU acceleration and device management
- Understand framework-specific optimization techniques
- Practice building neural network layers using these tensor operations

Both frameworks provide powerful tensor operations with similar capabilities. The choice often comes down to ecosystem preferences, deployment requirements, and team expertise rather than fundamental differences in tensor operations.